# Back to Julia

In [1]:
%load_ext Cython

This is what we had last time to start with...

## Pure python

In [2]:
from collections import namedtuple
Box = namedtuple("Box", "x1 x2 y1 y2")
bounds = Box(-1.8, 1.8, -1.8, 1.8)
focus=complex(-0.62772, -.42193)
gridsize=1000
iters=300

def setup_grid(gridsize, box):
    xstep = (box.x2 - box.x1)/(gridsize - 1.0)
    ystep = (box.y2 - box.y1)/(gridsize - 1.0)
    xs = (box.x1+ i* xstep for i in range(gridsize))
    zs=[]
    for x in xs:
        ys = (box.y1+ i* ystep for i in range(gridsize))
        for y in ys:
            zs.append(complex(x,y))
    return zs

def zts1(maxiter, zs, c): 
    output = [0] * len(zs)
    for i,z in enumerate(zs):
        n=0
        while n < maxiter and abs(z) < 2:
            z=z*z+c
            n+=1 
        output[i] = n
    return output

def run1():
    zs = setup_grid(gridsize, bounds)
    out = zts1(iters, zs, focus)
    return zs, out

In [3]:
%timeit -r 1 -n 5 run1()

5 loops, best of 1: 10.7 s per loop


And by the end of last lab you probably got to something like this...lets call it **B**

## cython

In [4]:
%%cython --annotate
cimport cython
@cython.boundscheck(False)
def zts1_cython4(int maxiter, zs, double complex c): 
    cdef unsigned int i, n
    cdef double complex z
    output = [0] * len(zs)
    i = 0
    for z in zs:
        n=0
        while n < maxiter and z.real*z.real + z.imag*z.imag < 4:
            z=z*z+c
            n+=1 
        output[i] = n
        i += 1
    return output

In [5]:
def run4():
    zs = setup_grid(gridsize, bounds)
    out = zts1_cython4(iters, zs, focus)
    return zs, out

In [6]:
%timeit -r 1 -n 5 run4()

5 loops, best of 1: 725 ms per loop


## Q1. 

Implement the zts kernel such that `zs` is treated as a memoryview coming in and output is `cdef`ed as a memoryview to a `zs` shaped empty numpy array.

In [19]:
%%cython --annotate
cimport cython
import numpy as np
cimport numpy as np
@cython.boundscheck(False)
#your code here
def zts1_numpy(int maxiter, double complex[:] zs, double complex c):
    cdef unsigned int i = 0
    cdef unsigned int n = 0
    cdef double complex z
    cdef int[:] output = np.empty(zs.shape[0], dtype = np.int32)
    for z in zs:
        n = 0
        while n < maxiter and z.real*z.real + z.imag * z.imag < 4:
            z = z * z + c
            n += 1
        output[i] = n
        i += 1
    return output

Now implement the grid zs as a numpy zeros array. Keep our generator based complex number creation going for now...

In [20]:
def setup_grid_numpy1(gridsize, box):
    #your code here
    startx = box.x1
    endx = box.x2
    starty = box.y1
    endy = box.y2
    xtick = (endx - startx)/np.float((gridsize - 1))
    ytick = (endy - starty)/np.float((gridsize - 1))
    xs = startx + np.arange(gridsize) * xtick
    output = np.zeros(gridsize * gridsize, dtype = np.complex128)
    ctr = 0
    for i,x in enumerate(xs):
        ys = starty + np.arange(gridsize) * ytick
        for j,y in enumerate(ys):
            output[i*len(ys)+j] = complex(x,y)
    return output

In [21]:
def run5():
    zs = setup_grid_numpy1(gridsize, bounds)
    out = zts1_numpy(iters, zs, focus)
    return zs, out

In [22]:
%timeit -r 1 -n 5 run5()

5 loops, best of 1: 1.06 s per loop


## using `np.meshgrid`

In [23]:
def setup_grid_numpy2(gridsize, box):
    xs = np.linspace(box.x1, box.x2, gridsize)
    ys = np.linspace(box.y1, box.y2, gridsize)
    X,Y = np.meshgrid(xs,ys)
    zs = X + 1j*Y
    return zs.reshape(gridsize*gridsize,)

def run6():
    zs = setup_grid_numpy2(gridsize, bounds)
    out = zts1_numpy(iters, zs, focus)
    return zs, out

In [24]:
%timeit -r 1 -n 5 run6()

5 loops, best of 1: 400 ms per loop


## Q2.

You'll notice that this intermediate `zs` array we created isnt actually needed. We could set up the `z`s, and immediately run the code which checks to see if the value of `z` escapes.  We have refactored some of this code for you below. Start with `setup_grid_numpy` and change it to create a `run_cython_amalgamated` function which computes the z and runs `escape` on it. This function should return a tuple of memoryviers `zs, output`.

The main loop should look like:
```python
for i in range(gridsize):
    x = xstart + i*xstep
    for j in range(gridsize):
        y = ystart + i*ystep
...
```

In [26]:
a = [1,2,3,4,6,7,8]
a[::2]

[1, 3, 6, 8]

In [28]:
%%cython --annotate
cimport cython
import numpy as np
cimport numpy as np

cdef inline double norm2(double complex z):
    return z.real*z.real + z.imag*z.imag

cdef int escape(int maxiter, double complex z, double complex c):
    cdef int n=0
    while n < maxiter and norm2(z) < 4:
        z=z*z+c
        n+=1
    return n
    
@cython.boundscheck(False)
@cython.wraparound(False)
def run_cython_amalgamated(int gridsize, box, double complex c, int maxiter):
    #your code here
    cdef int i,j
    cdef double startx = box.x1, starty = box.y1, endx = box.x2, endy = box.y2
    cdef unsigned int n
    cdef double xinit = 0., yinit = 0., x = 0., y = 0.
    cdef double complex z
    cdef double complex[:,:] zs = np.zeros((gridsize, gridsize), dtype = np.complex128)
    cdef int[:,:] output = np.empty_like(zs,dtype=np.int32)
    cdef double xtick = (endx - startx)/np.float((gridsize - 1))
    cdef double ytick = (endy - starty)/np.float((gridsize - 1))
    
    for i in np.arange(gridsize):
        x = xinit + i * xtick
        for j in np.arange(gridsize):
            y = yinit + j * ytick
            z = x + j * y
            zs[i,j] = z
            output[i,j] = escape(maxiter, z, c)
    return output


In [29]:
%timeit -r 1 -n 5 run_cython_amalgamated(gridsize, bounds, focus, iters)

5 loops, best of 1: 83.5 ms per loop


## Q3.

Lets nor parallelize this function. Cython allows us to do this using `openmp`, which is supported by the gcc, not the clang compiler. 

First notice that we have added `nogil` to the `cdef` functions that are called from main `def` function. Since those are in C, they can give up the GIL. So this is the unusual situation where coding in C allows us to get parallelism just using threads.

The main loop should look like:
```python
for i in prange(gridsize, nogil=True,
                    schedule='static', chunksize=1):
    x = xstart + i*xstep
    for j in range(gridsize):
        y = ystart + i*ystep
...
```

What's happening here is that `prange` partitions the outside loop to run on multiple threads in `chunksize` blocks, with a `static` schedule fixed at compile time. 

First write all the code in a `%%cython` cell below with annotations to see what it looks like.

In [31]:
%%cython --annotate
cimport cython
import numpy as np
cimport numpy as np
from cython.parallel cimport prange

cdef inline double norm2(double complex z) nogil:
    return z.real*z.real + z.imag*z.imag

cdef int escape(int maxiter, double complex z, double complex c) nogil:
    cdef int n=0
    while n < maxiter and norm2(z) < 4:
        z=z*z+c
        n+=1
    return n

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef run_cython_amalgamated2(int gridsize, box, double complex c, int maxiter):
    #your code here
    cdef int i,j
    cdef double startx = box.x1, starty = box.y1, endx = box.x2, endy = box.y2
    cdef unsigned int n
    cdef double xinit = 0., yinit = 0., x = 0., y = 0.
    cdef double complex z
    cdef double complex[:,:] zs = np.zeros((gridsize, gridsize), dtype = np.complex128)
    cdef int[:,:] output = np.empty_like(zs,dtype=np.int32)
    cdef double xtick = (endx - startx)/np.float((gridsize - 1))
    cdef double ytick = (endy - starty)/np.float((gridsize - 1))
    
    for i in prange(gridsize, nogil=True,schedule='static', chunksize=1):
        x = xinit + i * xtick
        for j in range(gridsize):
            y = yinit + j * ytick
            z = x + j * y
            zs[i,j] = z
            output[i,j] = escape(maxiter, z, c)
    return output



Then copy that all in a `parjulia.pyx` file with `openmp` compilation directives. To compile you will need gcc (`brew install gcc`), or a recent version of `clang`. Mine is 3.5 and openmp is not supported.

In [48]:
%%file parjulia.pyx
#distutils: extra_compile_args = -fopenmp
#distutils: extra_link_args = -fopenmp
cimport cython
import numpy as np
cimport numpy as np
from cython.parallel cimport prange

cdef inline double norm2(double complex z) nogil:
    return z.real*z.real + z.imag*z.imag

cdef int escape(int maxiter, double complex z, double complex c) nogil:
    cdef int n=0
    while n < maxiter and norm2(z) < 4:
        z=z*z+c
        n+=1
    return n

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef run_cython_amalgamated2(int gridsize, box, double complex c, int maxiter):
    #your code here
    cdef int i,j
    cdef double startx = box.x1, starty = box.y1, endx = box.x2, endy = box.y2
    cdef unsigned int n
    cdef double xinit = 0., yinit = 0., x = 0., y = 0.
    cdef double complex z
    cdef double complex[:,:] zs = np.zeros((gridsize, gridsize), dtype = np.complex128)
    cdef int[:,:] output = np.empty_like(zs,dtype=np.int32)
    cdef double xtick = (endx - startx)/np.float((gridsize - 1))
    cdef double ytick = (endy - starty)/np.float((gridsize - 1))
    
    for i in prange(gridsize, nogil=True,schedule='static', chunksize=1):
        x = xinit + i * xtick
        for j in range(gridsize):
            y = yinit + j * ytick
            z = x + j * y
            zs[i,j] = z
            output[i,j] = escape(maxiter, z, c)
    return output




Overwriting parjulia.pyx


Here is a `setup.py` to compile it

In [53]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize
import numpy

setup(name="parjulia",
      ext_modules=cythonize("parjulia.pyx"),
     include_dirs=[numpy.get_include()])

Overwriting setup.py


And this is what I did to build the module:

In [58]:
!export CC="/usr/local/bin/gcc-5"; /Users/yuhantang/anaconda/envs/py35/bin/python setup.py build_ext -if

running build_ext
building 'parjulia' extension
/usr/local/bin/gcc-5 -fno-strict-aliasing -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/yuhantang/anaconda/envs/py35/include -arch x86_64 -I/Users/yuhantang/anaconda/envs/py35/lib/python3.5/site-packages/numpy/core/include -I/Users/yuhantang/anaconda/envs/py35/include/python3.5m -c parjulia.c -o build/temp.macosx-10.5-x86_64-3.5/parjulia.o -fopenmp
In file included from /Users/yuhantang/anaconda/envs/py35/lib/python3.5/site-packages/numpy/core/include/numpy/ndarraytypes.h:1781:0,
                 from /Users/yuhantang/anaconda/envs/py35/lib/python3.5/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /Users/yuhantang/anaconda/envs/py35/lib/python3.5/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from parjulia.c:280:
/Users/yuhantang/anaconda/envs/py35/lib/python3.5/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: war

Lets run it!

In [59]:
from parjulia import run_cython_amalgamated2

In [60]:
%timeit -r 1 -n 5 run_cython_amalgamated2(gridsize, bounds, focus, iters)

5 loops, best of 1: 6.46 ms per loop
